In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("loans") \
.master("spark://10.208.36.84:7077") \
.config("spark.hadoop.defaultFS","hdfs://10.208.36.84:9000") \
.config("spark.executor.cores",2) \
.config("spark.executor.memory","4g") \
.config("spark.cores.max","8") \
.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 11:44:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
loschema = "loan_id string, total_principal_received float, total_interest_received float, total_late_fee_received float, total_payment_received float, last_payment_amount float, last_payment_date string, next_payment_date string,ingest_date date"


In [4]:
df = spark.read.format("csv").option("header","true") \
.schema(loschema)\
.load("/home/tushar/Documents/project/loans_repayment_raw")

In [5]:
df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



In [6]:
from pyspark.sql.functions import *

In [7]:
df = df.withColumn("ingest_date",current_timestamp())

In [8]:
df.createOrReplaceTempView("repay")

In [9]:
l = ["total_principal_received","total_interest_received","total_late_fee_received","total_payment_received","last_payment_amount"]

In [10]:
df = df.dropna(subset = l)

In [11]:
df.count()

868835

In [12]:
df.createOrReplaceTempView("loanrepay")

In [13]:
spark.sql("select * from loanrepay where total_principal_received = 0.0 ").show()

+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
| loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|68366599|                     0.0|                    0.0|                  43.27|               3638.63|              100.0|         Mar-2016|             null|2024-03-21 11:45:...|
|68393736|                     0.0|                    0.0|                    0.0|                   0.0|                0.0|             null|             null|2024-03-21 11:45:...|
|67255203|                     0.0|                    0.0|                    0

In [29]:
df.filter('total_principal_received = 0.0').count()

950

In [32]:
df.filter('next_payment_date is null ').show()

+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
| loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|68407277|                  3600.0|                 821.72|                    0.0|              4421.724|             122.67|         Jan-2019|             null|2024-03-19 15:16:...|
|68355089|                 24700.0|                 979.66|                    0.0|              25679.66|             926.35|         Jun-2016|             null|2024-03-19 15:16:...|
|68341763|                 20000.0|                2705.92|                    0

In [14]:
df.show()

+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
| loan_id|total_principal_received|total_interest_received|total_late_fee_received|total_payment_received|last_payment_amount|last_payment_date|next_payment_date|         ingest_date|
+--------+------------------------+-----------------------+-----------------------+----------------------+-------------------+-----------------+-----------------+--------------------+
|68407277|                  3600.0|                 821.72|                    0.0|              4421.724|             122.67|         Jan-2019|             null|2024-03-21 11:45:...|
|68355089|                 24700.0|                 979.66|                    0.0|              25679.66|             926.35|         Jun-2016|             null|2024-03-21 11:45:...|
|68341763|                 20000.0|                2705.92|                    0

In [15]:
df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [16]:
df.write.format("parquet").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_loan_repayment/parquet").save()

In [17]:
df.write.format("csv").mode("overwrite").option("path","/home/tushar/Documents/project/cleaned_loan_repayment/csv").save()